load project from github

In [ ]:
# Load DANN-DL github, populate necessary repositories
ls = !ls
if "CoRL_OODWorkshop_DANN-DL" in ls[0]:
  None
else:
  !git clone https://github.com/StanfordASL/CoRL_OODWorkshop_DANN-DL.git
  !cd CoRL_OODWorkshop_DANN-DL/Modules && git clone https://github.com/tadeephuy/GradientReversal.git
  !cd CoRL_OODWorkshop_DANN-DL/Modules && git clone https://github.com/StanfordASL/scod-module
  !cd CoRL_OODWorkshop_DANN-DL/Modules && mv ./scod-module ./scod_module

library import

In [ ]:
import torch
import torchvision
import os
import sys

pwd = !pwd
project_dir = os.path.join(pwd[0], "CoRL_OODWorkshop_DANN-DL")
sys.path.append(project_dir)

from Modules.DANN.DANN import DANN, DA_parameter
from Modules.CNN.CNN_MNIST import CNN_MNIST
from Modules.DANN.DANN_train import DANN_train
from Modules.DANN.DANN_eval import DANN_eval
from get_device import get_device

from Modules.scod_module.scod.scod import SCOD as SCOD_CNN
from Modules.scod_module.scod.scod import OodDetector
from Modules.scod_module.scod.distributions import CategoricalLogitLayer

from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import MNIST, ImageFolder
from torchvision import transforms
from tqdm import trange

meta data

In [ ]:
# Hyperparameters
epochs = 100
batch_size = 64
lr = 1e-4
domain_loss_weight = 0.1

# Training set
source_train_size = 11000
target_train_size = 0 # no target data at initial deployment -- we don't know what we'll see
image_size = 28

# Deployment
num_episodes = 10
images_per_episode = 75
percent_OOD = 0.2 # 0.05
scod_percentile = 0.95 # OOD threshold (on source)

print("Episodes: ", num_episodes)
print("Percent data is OOD: ", percent_OOD, "\n")

# Run chacterization
run_name = "local_kl" # SCOD method
assert run_name == "local_kl" or run_name == "entropy" or run_name == "var"

load datasets

In [ ]:
mnist_transform = transforms.Compose([
    transforms.ToTensor()
])
mnistm_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(image_size)
])

_train_source = MNIST(root = os.path.join(project_dir, 'MNIST/train'), train = True, download = True, transform = mnist_transform)
_train_target = ImageFolder(os.path.join(project_dir, 'MNIST-M/training'), transform = mnistm_transform)

train_source, _deploy_source, _ = torch.utils.data.random_split(_train_source, lengths = [source_train_size, int(images_per_episode*(1-percent_OOD))*num_episodes, len(_train_source) - source_train_size - int(images_per_episode*(1-percent_OOD))*num_episodes])
train_target, _deploy_target, _ = torch.utils.data.random_split(_train_target, lengths = [target_train_size, int(images_per_episode*percent_OOD)*num_episodes, len(_train_target) - target_train_size - int(images_per_episode*percent_OOD)*num_episodes])

deploy_sources = torch.utils.data.random_split(_deploy_source, lengths = [1/num_episodes]*num_episodes)
deploy_targets = torch.utils.data.random_split(_deploy_target, lengths = [1/num_episodes]*num_episodes)

test_source = MNIST(root = os.path.join(project_dir, "MNIST/test"), train = False, download = True, transform = mnist_transform)
test_target = ImageFolder(os.path.join(project_dir, "MNIST-M/testing"), transform = mnistm_transform)

print("Source training set size: ", len(train_source))
print("Source images available for deployment: ", int(images_per_episode*(1-percent_OOD))*num_episodes)
print("Source images per episode: ", len(deploy_sources[0]))

print("Target training set size: ", len(train_target))
print("Target images available for deployment: ", int(images_per_episode*percent_OOD)*num_episodes)
print("Target images per episode: ", len(deploy_targets[0]))

print("Source test set size: ", len(test_source))
print("Target test set size: ", len(test_target), '\n')

device definition

In [ ]:
device = get_device()

incorporate scod object

In [6]:
def build_scod(run_name, model, train_source):
  args = {
    'num_eigs': 100,
    'num_samples': 304,
    'sketch_type': 'srft'
  }

  # only use SCOD from CNN
  scod_model = SCOD_CNN(model, args = args)

  dist_layer = CategoricalLogitLayer()
  scod_model.process_dataset(train_source, dist_layer)
  ood_detector = OodDetector(scod_model, dist_layer, run_name)

  return ood_detector

def collate_fn(batch):
  return torch.stack([x[0].expand(3, -1, -1) for x in batch]), torch.hstack([torch.tensor([x[1]]) for x in batch])

def perform_detection(ood_detector, deploy_source, deploy_target, source, percentile):
  batch_size = 64
  deploy_source_dataloader = torch.utils.data.DataLoader(deploy_source, batch_size = batch_size)
  deploy_target_dataloader = torch.utils.data.DataLoader(deploy_target, batch_size = batch_size)
  source_dataloader = torch.utils.data.DataLoader(source, batch_size = batch_size, collate_fn = collate_fn)

  scod_source = torch.cat([ood_detector(batch[0].to(device)).detach().cpu() for batch in deploy_source_dataloader])
  scod_target = torch.cat([ood_detector(batch[0].to(device)).detach().cpu() for batch in deploy_target_dataloader])
  scod_train_source = torch.cat([ood_detector(batch[0].to(device)).detach().cpu() for batch in source_dataloader])

  ood_source = torch.utils.data.Subset(deploy_source, torch.where(scod_source > torch.quantile(scod_train_source, q = percentile))[0])
  ood_target = torch.utils.data.Subset(deploy_target, torch.where(scod_target > torch.quantile(scod_train_source, q = percentile))[0])

  return ood_source, ood_target

simulate deployment

In [ ]:
# First episode CNN deployment
episode = 0
model = CNN_MNIST()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)
ce_loss = torch.nn.CrossEntropyLoss(reduction = "sum")

# train CNN classifier
mnist_dataloader = torch.utils.data.DataLoader(train_source, batch_size = batch_size, shuffle = True, pin_memory = True)
writer = SummaryWriter("./CoRL_OODWorkshop_DANN-DL/Tensorboard/" + run_name + "/episode" + str(episode))

for epoch in trange(100):
  epoch_loss = 0.
  for ip, op in mnist_dataloader:
      ip, op = ip.to(device), op.to(device) # transfer ip/op to GPU, if applicable
      pred = model(ip)

      optimizer.zero_grad()
      loss = ce_loss(pred, op)
      mean_loss = loss / ip.shape[0]
      mean_loss.backward()

      optimizer.step()
      epoch_loss += loss.item()
  epoch_loss /= len(train_source)
  scheduler.step()
  writer.add_scalar("mean_epoch_loss", epoch_loss, epoch)

torch.save(model, "./CoRL_OODWorkshop_DANN-DL/Tensorboard/" + run_name + "/episode" + str(episode) + "/episode" + str(episode) + "_model.pth")
writer.flush()

# Evaluate CNN
softmax = torch.nn.Softmax(dim = -1)
test_mnist_dataloader = torch.utils.data.DataLoader(test_source, batch_size = batch_size)
test_mnistm_dataloader = torch.utils.data.DataLoader(test_target, batch_size = batch_size)

mnist_correct = 0
mnistm_correct = 0

for ip, op in test_mnist_dataloader:
    ip, op = ip.to(device), op.to(device)

    pred_logits = model(ip)
    pred_softmax = softmax(pred_logits)
    pred = torch.argmax(pred_softmax, dim = -1)

    mnist_correct += len(torch.where(pred == op)[0])

for ip, op in test_mnistm_dataloader:
    ip, op = ip.to(device), op.to(device)

    pred_logits = model(ip)
    pred_softmax = softmax(pred_logits)
    pred = torch.argmax(pred_softmax, dim = -1)

    mnistm_correct += len(torch.where(pred == op)[0])

print("CNN MNIST Accuracy: ", mnist_correct / len(test_source))
print("CNN MNIST-M Accuracy: ", mnistm_correct / len(test_target))

writer.add_scalar("source_acc", mnist_correct / len(test_source), episode)
writer.add_scalar("target_acc", mnistm_correct / len(test_target), episode)
writer.add_scalar("source_training_size", len(train_source), episode)
writer.add_scalar("target_training_size", len(train_target), episode)
writer.flush()
writer.close()

In [ ]:
print("--- " + run_name + " ---")

ood_detector = build_scod(run_name, model, train_source)

# Build DANN architecture after first deployment
for episode in range(1, num_episodes+1):

  # perform OOD detection with previous model
  ood_source, ood_target = perform_detection(
      ood_detector,
      deploy_sources[episode-1],
      deploy_targets[episode-1],
      train_source,
      scod_percentile)
  if len(train_source) < len(train_target) + len(ood_source) + len(ood_target):
    break # stop after target larger than source

  dirpath = "./CoRL_OODWorkshop_DANN-DL/Tensorboard/" + run_name + "/episode" + str(episode)
  writer = SummaryWriter(dirpath)

  # train model
  model, train_target, writer = DANN_train(
      train_source,
      train_target,
      torch.utils.data.ConcatDataset((ood_source, ood_target)),
      test_target,
      (epochs, lr, batch_size, 0, domain_loss_weight),
      (device, writer)
    )
  torch.save(model, dirpath + "/episode" + str(episode) + "_model.pth")

  # evaluate model
  source_acc, target_acc = DANN_eval(model, test_source, test_target, (batch_size, 0), device)
  writer.add_scalar("source_acc", source_acc, episode)
  writer.add_scalar("target_acc", target_acc, episode)

  writer.add_scalar("source_training_size", len(train_source), episode)
  writer.add_scalar("target_training_size", len(train_target), episode)

  writer.add_scalar("false_positive_rate", len(ood_source)/len(deploy_sources[episode-1]), episode) # store false positives
  writer.add_scalar("true_positive_rate", len(ood_target)/len(deploy_targets[episode-1]), episode) # store true negatives

  writer.flush()
  writer.close()

  print("Episode ", episode)
  print("\tSource acc: ", source_acc)
  print("\tTarget acc: ", target_acc)
  print("\tTarget training size: ", len(train_target))
  print("\t\tfalse positive rate: ", len(ood_source)/len(deploy_sources[episode-1]))
  print("\t\ttrue positive rate: ", len(ood_target)/len(deploy_targets[episode-1]))

In [ ]:
!zip -r ./results.zip ./CoRL_OODWorkshop_DANN-DL/Tensorboard/local_kl/

In [ ]:
from google.colab import files
files.download("./results.zip")